<img alt='UCL' src="images/ucl_logo.png" align='center'>


[<img src="images/noun_post_2109127.svg" width="50" align='right'>](016_Python_for.ipynb)
[<img src="images/noun_pre_2109128.svg" width="50" align='right'>](018_Python_xxx.ipynb)



# 021 Streams


## Introduction


### Purpose

In this session, we will learn how to read files and similar resources. We will mainly use [`pathlib`](https://docs.python.org/3/library/pathlib.html) and the local package [gurlpath](geog0111/gurlpath) derived from [`urlpath`](https://github.com/chrono-meter/urlpath). We will also cover opening and closing files, and some simple read- and write-operations.


### Prerequisites

You will need some understanding of the following:


* [001 Using Notebooks](001_Notebook_use.ipynb)
* [002 Unix](002_Unix.ipynb) with a good familiarity with the UNIX commands we have been through.
* [003 Getting help](003_Help.ipynb)
* [010 Variables, comments and print()](010_Python_Introduction.ipynb)
* [011 Data types](011_Python_data_types.ipynb) 
* [012 String formatting](012_Python_strings.ipynb)
* [013_Python_string_methods](013_Python_string_methods.ipynb)
* [020_Python_files](020_Python_files.ipynb)

You will need to recall details from [020_Python_files](020_Python_files.ipynb) on using the two packages.


## Reading and writing

We can conveniently use `pathlib` to deal with file input and output. The main methods to be aware of are:


|command|  purpose|
|---|---|
|`Path.open()`| open a file and return a file descriptor|
|`Path.read_text()`|  read text|
|`Path.write_text()`| write text|
|`Path.read_bytes()`| read byte data|
|`Path.write_bytes()`| write byte data|


For `gurlpath` we have the following equivalent functions:





|command|  purpose|
|---|---|
|`URL.open()`| open a file descriptor with data from a URL|
|`URL.read_text()`|  read text from URL|
|`URL.write_text()`| write text to file|
|`URL.read_bytes()`| read byte data from URL|
|`URL.write_bytes()`| write byte data to file|

Notice that the `write` functions (and `open` when used for write) write to local files, not to the URL. 


### `with ... as ...`, `Path.open`, `URL.open`

Sometimes, we pass a filename or URL to some reading routine. But other times, we need to pass a stream. A [stream](https://en.wikipedia.org/wiki/Standard_streams) is a channel through which we may send and/or receive information. This is different to a file, which is where information may reside, but we may for instance open a stream to write to a file. In Python, we call the object that we get when opening a stream a *file object*.

A example of when we would use a stream is for instance when we want to take some information from a URL and pass it directly on to some Python function. A more long-winded way of doing that would be to save the information from the URL into a file on the local file system, then to read from that file into the function. Using a stream, we avoid the need to save a file. 

Of course sometimes, it may be convenient to store a temporary file for such a process, especially if we might want to re-use the file information. This is called building a **cache**. Each time we try to pull data from the original stream then, we would instead read from the local cache. You will notice that web browsers make extensive use of such ideas in trying to speed up the display of web pages: they try only to pull a new version of some data if it has changed.

The `pathlib` function for opening a stream is `Path.open`, with `URL.open` the corresponding function for URLs in our library.

The usual way of opening a file (or URL) to get the file object is:

    with Path(filename).open('r') as f:
       # do some reading with f
       pass
       

We use the form `with ... as ...` here, so that the file object `f` only exists within this construct and the file is automatically closed when we finish. Codes are spaced in inside the construct, as we have seen in `if ...` or `for ... in ...` constructs.

We have set the flag `r` within the `open()` function (this is the default mode). This means that the file will be opened for *reading* only. Alternatives include `w` for writing, or `w+` for appending.


### Using streams with `yaml`, `json`

Two common text formats for certain types of data representation are [json](https://docs.python.org/3/library/json.html) and [`yaml`](http://zetcode.com/python/yaml/). The Python library functions for input and output of both of these use streams: `yaml.safe_load()`, `yaml.safe_dump()`, `json.load()` and `json.dump()` respectively.

In [1]:
import yaml

help(yaml.safe_load)
help(yaml.safe_dump)

Help on function safe_load in module yaml:

safe_load(stream)
    Parse the first YAML document in a stream
    and produce the corresponding Python object.
    
    Resolve only basic YAML tags. This is known
    to be safe for untrusted input.

Help on function safe_dump in module yaml:

safe_dump(data, stream=None, **kwds)
    Serialize a Python object into a YAML stream.
    Produce only basic YAML tags.
    If stream is None, return the produced string instead.



In the following example, we use `Path` to open the file [`bin/copy/environment.yml`](bin/copy/environment.yml) and read the open stream using the `yaml.safe_load()` function. 

This file, [`bin/copy/environment.yml`](bin/copy/environment.yml), specifies which packages are loaded in our Python environment. It has a simple ASCII format, but since it is a `yaml` file, we should read it with code that interprets the format correctly and safely into a dictionary. 

In [2]:
from pathlib import Path
import yaml

# form the file name
yaml_file = Path('bin/copy/environment.yml')

# open stream object 'read'
with yaml_file.open('r') as f:
    env = yaml.safe_load(f)

print(f'env is type {type(env)}')
print(f'env keys: {env.keys()}')

env is type <class 'dict'>
env keys: dict_keys(['name', 'channels', 'dependencies'])


We can access this same file from a URL in the course code repository on GitHub. The  equivalent, reading the data from a URL is:

In [3]:
from geog0111.gurlpath import URL
import yaml

# form the file name
site = 'https://raw.githubusercontent.com'
site_dir = '/UCL-EO/geog0111/master'
site_file = 'copy/environment.yml'
yaml_file = URL(site,site_dir,site_file,verbose=True)

# notice that we can use verbose=True for URL open
with yaml_file.open('r') as f:
    env = yaml.safe_load(f)

print(f'env is type {type(env)}')
print(f'env keys: {env.keys()}')

--> opening stream to https://raw.githubusercontent.com/UCL-EO/geog0111/master/copy/environment.yml:
--> reading data from https://raw.githubusercontent.com/UCL-EO/geog0111/master/copy/environment.yml
--> open() text stream
--> trying https://raw.githubusercontent.com/UCL-EO/geog0111/master/copy/environment.yml


env is type <class 'dict'>
env keys: dict_keys(['name', 'channels', 'dependencies'])


We can similarly use a stream to write the information in `env` into a `json` format file:

In [4]:
from pathlib import Path
import json

# form the file name
json_file = Path('bin/copy/environment.json')

with json_file.open('w') as f:
    json.dump(env, f)
    

#### Exercise 1

* write code to read from the json-format file `bin/copy/environment.json` into a dictionary called `json_data`.
* print out the dictionary keys.
* print the file size of the json-format file in KB to two decimal places.

In [5]:
# ANSWER
# write code to read from the json-format file 
# bin/copy/environment.json 
# into a dictionary called json_data.
json_file = Path('bin/copy/environment.json')

# use with ... as ... as we have been shown
with json_file.open('r') as f:
    json_data = json.load(f)
    
# print out the dictionary keys.
print(json_data.keys())

# print the file size of the 
# json-format file in KB to two decimal places.
print(f'file {json_file} size {json_file.stat().st_size / 1024 : .2f}')

dict_keys(['name', 'channels', 'dependencies'])
file bin/copy/environment.json size  0.78


### Reading data into `pandas`

The Python package `pandas` is widely-used to read and manipulate data in tabular or similar form. One of the most common tabular data formats is [CSV](https://en.wikipedia.org/wiki/Comma-separated_values).

An interesting CSV-format dataset is that containing the [successive pulses](https://gist.githubusercontent.com/borgar/31c1e476b8e92a11d7e9/raw/0fae97dab6830ecee185a63c1cee0008f6778ff6/pulsar.csv) of the oscillation signal coming from the [Pulsar PSR B1919+21](https://www.joydivisionofficial.com/reimagined/) discovered by [Jocelyn Bell](https://en.wikipedia.org/wiki/Jocelyn_Bell_Burnell) in 1967.

![Joy Division](images/unknown_pleasures.png)

[By inspection](https://raw.githubusercontent.com/igorol/unknown_pleasures_plot/master/pulsar.csv) we can see the data are 80 lines of 300 columns of data. The data format is simple, with no missing values or metadata. We can straightforwardly use the `pandas` function `pd.read_csv`, specifying the URL, to read this dataset (specifying only `header=None` so that the first line is not interpreted as data column names).

In [6]:
import pandas as pd
from geog0111.gurlpath import URL

site = 'https://raw.githubusercontent.com'
site_dir = 'igorol/unknown_pleasures_plot/master'
site_file = 'pulsar.csv'

url = URL(site,site_dir,site_file)

df=pd.read_csv(url,header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.81,-0.91,-1.09,-1.00,-0.59,-0.82,-0.43,-0.68,-0.71,-0.27,...,-0.08,0.19,-0.19,-0.18,-0.20,-0.26,-0.52,-0.44,-0.58,-0.54
1,-0.61,-0.40,-0.42,-0.38,-0.55,-0.51,-0.71,-0.79,-0.52,-0.40,...,-0.34,-0.58,-0.26,-0.64,-1.05,-0.83,-0.80,-0.47,-0.13,-0.12
2,-1.43,-1.15,-1.25,-1.13,-0.76,-0.25,0.40,0.26,0.30,0.36,...,-0.29,0.16,0.83,0.99,1.28,0.11,-0.77,-0.88,-0.45,-1.01
3,-1.09,-0.85,-0.72,-0.74,-0.26,-0.04,-0.19,0.18,0.03,0.19,...,0.48,0.52,-0.14,-1.13,-1.07,-1.03,-0.78,-0.40,0.18,0.27
4,-1.13,-0.98,-0.93,-0.90,-1.14,-1.00,-0.90,-1.18,-1.30,-1.07,...,-0.27,-0.47,-0.49,-0.23,-0.75,-0.29,-0.54,-0.65,-0.64,-0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.62,0.64,0.59,0.30,0.01,0.05,-0.63,0.07,0.36,0.78,...,0.20,0.22,0.23,0.27,-0.10,-0.21,-0.09,-0.24,-0.17,-0.62
76,0.32,0.31,0.28,0.42,-0.24,-0.48,-0.73,-0.64,0.04,0.02,...,-0.44,-0.53,-0.50,-0.49,-0.63,-0.56,-0.50,-0.38,-0.58,-0.43
77,-0.09,-0.14,-0.24,-0.24,-0.66,0.00,0.29,0.29,0.60,0.86,...,0.08,-0.88,-1.17,-0.36,-0.31,-0.12,0.29,-0.02,0.21,0.44
78,0.11,0.05,0.05,-0.05,-0.03,-0.29,-0.08,-0.54,-0.01,0.01,...,-0.73,-0.54,-0.53,-0.92,-0.68,-0.87,-1.31,-1.02,-1.10,-1.62


Not all data files we find on the web may be so straightforward to read though. In [020_Python_files](020_Python_files.ipynb) we saw data of Monthly Southeast England precipitation (mm) in a tabular form on the [Met Office website](https://www.metoffice.gov.uk/hadobs/hadukp/data/monthly/HadSEEP_monthly_qc.txt). We would commonly use the Python package [`pandas`](https://pandas.pydata.org/) to read and analyses such data. 

A more general and reliable method with `pandas` then is to provide a stream to read from. We demonstrate that here with the Met Office data of . [By inspection](https://www.metoffice.gov.uk/hadobs/hadukp/data/monthly/HadSEEP_monthly_qc.txt), the dataset is seen to have a variable number of spaces between the data columns. This is known as 'whitespace' (i.e. ` ` or `\t` characters). This makes it more complex to read into `pandas` than a CSV format, and we need to specify a [regular expression](https://en.wikipedia.org/wiki/Regular_expression) meaning 'one or more space'. This is `r"[ ]{1,}"` and we give the keyword `sep` for `pandas` as `sep=r"[ ]{1,}"`. Further for `pandas` in this case we must specify that we should use the Python engine to interpret `engine='python'`. Other features of the dataset are that the first 3 rows of data are metadata and should be skipped in reading the dataset: `skiprows=3`, with the 4th line the data column headers. Finally, we see that 'no data' values are given here as the value `-99.9`: `na_values=[-99.9]`. 

Since there are quite a few keyword options to use, we might find it convenient to gather these into a dictionary:

    panda_format = {
        'skiprows'   :  3,
        'na_values'  :  [-99.9],
        'sep'        :  r"[ ]{1,}",
        'engine'     :  'python'
    }


With these file-formatting specifications, we can read this dataset directly into a `pandas` data frame using a stream that we open from the URL with:

    url.open('r')

In [7]:
import pandas as pd
from geog0111.gurlpath import URL

# NB -- avoid trailing / on these
# i.e. dont put 
# site_dir = 'hadobs/hadukp/data/monthly/'
site = 'https://www.metoffice.gov.uk/'
site_dir = 'hadobs/hadukp/data/monthly'
site_file = 'HadSEEP_monthly_qc.txt'

url = URL(site,site_dir,site_file)

panda_format = {
    'skiprows'   :  3,
    'na_values'  :  [-99.9],
    'sep'        :  r"[ ]{1,}",
    'engine'     :  'python'
}

df=pd.read_table(url.open('r'),**panda_format)

# df.head: first n lines
df.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,1873,87.1,50.4,52.9,19.9,41.1,63.6,53.2,56.4,62.0,86.0,59.4,15.7,647.7
1,1874,46.8,44.9,15.8,48.4,24.1,49.9,28.3,43.6,79.4,96.1,63.9,52.3,593.5
2,1875,96.9,39.7,22.9,37.0,39.1,76.1,125.1,40.8,54.7,137.7,106.4,27.1,803.5
3,1876,31.8,71.9,79.5,63.6,16.5,37.2,22.3,66.3,118.2,34.1,89.0,162.9,793.3
4,1877,146.0,47.7,56.2,66.4,62.3,24.9,78.5,82.4,38.4,58.1,144.5,54.2,859.6


#### Exercise 2

The file [2276931.csv](https://raw.githubusercontent.com/UCL-EO/geog0111/master/data/2276931.csv) contains precipitation data for an [NOAA weather station](https://www.ncdc.noaa.gov/cdo-web/datasets#GSOY) `HAVANA 4.2 SW, FL US` for the year 2020 to date.

The dataset URL is:

https://raw.githubusercontent.com/UCL-EO/geog0111/master/data/2276931.csv

* Inspect the file to discover any issues you must account for.
* Read the file into `pandas` using `url.open('r')`.
* print the first 5 lines of data

In [8]:
# ANSWER
msg = '''
Inspect the file to discover any issues you must account for.

The file is straightforward CVS format, with the first column
the data column titles
'''
print(msg)

import pandas as pd
from geog0111.gurlpath import URL

site = 'https://raw.githubusercontent.com'
site_dir = '/UCL-EO/geog0111/master/data'
site_file = '2276931.csv'

# form the URL
url = URL(site,site_dir,site_file)

# Read the file into pandas using url.open('r').
df=pd.read_csv(url.open('r'))

# print the first 5 lines of data
df.head(5)


Inspect the file to discover any issues you must account for.

The file is straightforward CVS format, with the first column
the data column titles



,STATION,NAME,DATE,PRCP,SNOW
0,US1FLGD0002,"HAVANA 4.2 SW, FL US",2020-01-01,0.00,0.0
1,US1FLGD0002,"HAVANA 4.2 SW, FL US",2020-01-02,0.00,0.0
2,US1FLGD0002,"HAVANA 4.2 SW, FL US",2020-01-03,0.00,0.0
3,US1FLGD0002,"HAVANA 4.2 SW, FL US",2020-01-04,0.98,NaN
4,US1FLGD0002,"HAVANA 4.2 SW, FL US",2020-01-05,0.00,0.0


In [9]:
# ANSWER 

# print out the absolute pathname of the 
# directory that images/ucl.png is in
abs_name = Path('images/ucl.png').absolute()
print(abs_name)

# we want the parent!
print(f'the file {abs_name.name} is in {abs_name.parent}')

# print the size of the file in bytes without reading the datafile. 
print(f'{abs_name.name} has size {abs_name.stat().st_size} bytes')

# 1 KB is 1024 Bytes
# .2f is 2 d.p. format
print(f'{abs_name.name} has size ' +\
      f'{abs_name.stat().st_size/1024:.2f} KB')

# read the datafile, and check you get the same data size
dataset = abs_name.read_bytes()
# size
s = len(dataset)
print(f'the size of data read is {s} bytes -> {s/1024 : .2f} KB')

/Users/plewis/Documents/GitHub/geog0111/notebooks/images/ucl.png
the file ucl.png is in /Users/plewis/Documents/GitHub/geog0111/notebooks/images
ucl.png has size 1956 bytes
ucl.png has size 1.91 KB
the size of data read is 1956 bytes ->  1.91 KB


## Summary

In this section, we have used `Path` and `URL` classes to open streams from files and URLs. We have seen how to use this to read the stream into packages that can interpret various formats of data, such as `yaml`, `json`, and CSV and tabular data. 

We have seen that using these object-oriented classes to deal with files and URLs means that we can use essentially the same functions throughout. 

We have come across the `pandas` package for reading tabular and similar datasets.


[<img src="images/noun_post_2109127.svg" width="50" align='right'>](016_Python_for.ipynb)
[<img src="images/noun_pre_2109128.svg" width="50" align='right'>](014_Python_groups.ipynb)
